In [2]:
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import pandas as pd
from tensorflow import keras
import torch
import keras.backend as K 
from keras.utils import to_categorical
from keras.models import load_model
from tifffile import imread, imwrite

2022-09-28 20:54:19.410116: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-28 20:54:19.410153: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/jacob-relle/miniconda3/envs/aml_projekt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Hyperparameter
img_size=400
patch_size = img_size

rings = [ 0.,  1.,  2.,  3.]
num_ring_classes = len(rings)
ring_classes = np.arange(num_ring_classes)

wedges = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,25]
num_wedge_classes = len(wedges) 
wedge_classes = np.arange(num_wedge_classes)


In [4]:
#Loss Function
def combo_loss_ring(targets, prediction):
    
    intersection = K.sum(tf.cast(targets*prediction,tf.float32),axis=(1,2))
    union = (K.sum(targets,axis=(1,2)) + K.sum(prediction,axis=(1,2)))
    dice = 1 - (2./num_ring_classes)*K.sum(intersection/union,axis=1)
    crossentropy = keras.losses.CategoricalCrossentropy()(targets,prediction)

    return dice + crossentropy

def combo_loss_wedge(targets, prediction):
    
    intersection = K.sum(tf.cast(targets*prediction,tf.float32),axis=(1,2))
    union = (K.sum(targets,axis=(1,2)) + K.sum(prediction,axis=(1,2)))
    dice = 1 - (2./num_wedge_classes)*K.sum(intersection/union,axis=1)
    crossentropy = keras.losses.CategoricalCrossentropy()(targets,prediction)

    return dice + crossentropy


def load_rings(label_name):
    # append label
    labels = np.zeros((1,patch_size,patch_size,num_ring_classes))
    label = cv2.resize(imread(label_name),(img_size,img_size),interpolation = cv2.INTER_NEAREST)
    label = to_categorical(label,num_ring_classes)
    labels[0] = label
    return labels

def load_wedges(label_name):
    # append label
    labels = np.zeros((1,patch_size,patch_size,num_wedge_classes))
    label = cv2.resize(imread(label_name),(img_size,img_size),interpolation = cv2.INTER_NEAREST)
    for i, score in enumerate(wedges):
        label[label == score] = i
    label = to_categorical(label,num_wedge_classes)
    labels[0] = label
    return labels

def load_img(img_name):
    # append image
    images = np.zeros((1,patch_size,patch_size,3))
    new_img = cv2.resize(cv2.imread(img_name),(400,400)).astype(np.float32)
    new_img -= np.mean(new_img)
    new_img /= np.var(new_img)
    images[0] = new_img

    return images

In [5]:
#Read the data
dataframe = pd.read_pickle(f'/home/jacob-relle/Dokumente/deepdart_aml/labelswithscore.pkl')

d1_test = ['d1_03_03_2020', 'd1_03_19_2020', 'd1_03_23_2020', 'd1_03_27_2020', 'd1_03_28_2020', 'd1_03_30_2020', 'd1_03_31_2020']
d2_test = ['d2_03_03_2020', 'd2_02_10_2021', 'd2_02_03_2021_2']
test = d1_test + d2_test
test_samples = 2150

#models
ringNet = load_model('ringNet.h5')
ringNet.compile(loss=combo_loss_ring, metrics='acc')
wedgeNet = load_model('wedgeNet.h5')
wedgeNet.compile(loss=combo_loss_wedge, metrics='acc')
dartNet = torch.hub.load('ultralytics/yolov5','custom','/home/jacob-relle/Dokumente/AML Projekt/exp4/weights/last.pt')

2022-09-28 20:54:35.215154: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-28 20:54:35.215203: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Murasame): /proc/driver/nvidia/version does not exist
2022-09-28 20:54:35.218506: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using cache found in /home/jacob-relle/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-9-28 Python-3.9.12 torch-1.12.1 CPU



[Errno 2] No such file or directory: '/home/jacob-relle/miniconda3/envs/aml_projekt/lib/python3.9/site-packages/numpy-1.23.1.dist-info/METADATA'


Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


In [42]:
ring_acc = 0
ring_var = 0
ring_top = 0
ring_bottom = 1
wedge_acc = 0
wedge_var = 0
wedge_top = 0
wedge_bottom = 1
for i in range(len(dataframe)):
    if dataframe["img_folder"][i] in test:
        if 'DSC' in dataframe['img_name'][i]:
            group = 'test'
            score = dataframe['score'][i]

            #load the data to test and validate
            image = load_img(f'/home/jacob-relle/Dokumente/AML Projekt/cropped_images/800/{group}/{dataframe["img_folder"][i]}/{dataframe["img_name"][i]}')
            ring_label = load_rings(os.path.splitext(f'/home/jacob-relle/Dokumente/AML Projekt/deepdart_data_ring/{group}/{dataframe["img_name"][i]}')[0] + '.tif')
            wedge_label = load_wedges(os.path.splitext(f'/home/jacob-relle/Dokumente/AML Projekt/deepdart_data_wedge/{group}/{dataframe["img_name"][i]}')[0] + '.tif')

            y_ring = ringNet.evaluate(image,ring_label,verbose=0)[1]
            ring_acc += y_ring
            ring_var += (y_ring - 0.959526)**2
            ring_top = max(ring_top,y_ring)
            ring_bottom = min(ring_bottom,y_ring)

            y_wedge = wedgeNet.evaluate(image,wedge_label,verbose=0)[1]
            wedge_acc += y_wedge
            wedge_var += (y_wedge - 0.986520)**2
            wedge_top = max(wedge_top,y_wedge)
            wedge_bottom = min(wedge_bottom,y_wedge)
            # Log every 5 batches.
    if i % 100 == 0:
        print(f"Progress: {(i + 1)}")
print(f' Ring acc: {ring_acc/150},Variance: {ring_var/150}, top:{ring_top}, bottom{ring_bottom}')
print(f'Wedge acc: {wedge_acc/150}, Variance: {wedge_var/150}, top:{wedge_top}, bottom:{wedge_bottom}')

Progress: 1
Progress: 101
Progress: 201
Progress: 301
Progress: 401
Progress: 501
Progress: 601
Progress: 701
Progress: 801
Progress: 901
Progress: 1001
Progress: 1101
Progress: 1201
Progress: 1301
Progress: 1401
Progress: 1501
Progress: 1601
Progress: 1701
Progress: 1801
Progress: 1901
Progress: 2001
Progress: 2101
Progress: 2201
Progress: 2301
Progress: 2401
Progress: 2501
Progress: 2601
Progress: 2701
Progress: 2801
Progress: 2901
Progress: 3001
Progress: 3101
Progress: 3201
Progress: 3301
Progress: 3401
Progress: 3501
Progress: 3601
Progress: 3701
Progress: 3801
Progress: 3901
Progress: 4001
Progress: 4101
Progress: 4201
Progress: 4301
Progress: 4401
Progress: 4501
Progress: 4601
Progress: 4701
Progress: 4801
Progress: 4901
Progress: 5001
Progress: 5101
Progress: 5201
Progress: 5301
Progress: 5401
Progress: 5501
Progress: 5601
Progress: 5701
Progress: 5801
Progress: 5901
Progress: 6001
Progress: 6101
Progress: 6201
Progress: 6301
Progress: 6401
Progress: 6501
Progress: 6601
Progres

In [6]:
#score_list = []
#predicted_score_list = []

for i in range(len(dataframe)):
    if dataframe["img_folder"][i] in test:
        if predicted_score_list[i]!=score_list:
            group = 'test'
            score = dataframe['score'][i]
            predicted_score = 0

            image_path = f'/home/jacob-relle/Dokumente/AML Projekt/cropped_images/800/{group}/{dataframe["img_folder"][i]}/{dataframe["img_name"][i]}'

            #load the data to test and validate
            image = load_img(image_path)
            ring = ringNet.predict(image,verbose=0)
            ring = np.argmax(ring,axis=3)

            wedge = wedgeNet.predict(image,verbose=0)
            wedge = np.argmax(wedge,axis=3)
            wedge[wedge==21] = 25
            
            total = ring*wedge
            total[total == 75] = 50
            
            dartNet_result = dartNet(image_path)
            darts = dartNet_result.xyxyn[0][:,:-1].numpy()
            for j in range(min(darts.shape[0],3)):
                if darts[j,-1] > 0.5:
                    dart_pos = (int(400*(darts[j,3] + darts[j,1]) /2) , int(400*(darts[j,2] + darts[j,0]) /2))
                    predicted_score += total[0,int(dart_pos[0]),int(dart_pos[1])]
                    total[0,int(dart_pos[0]),int(dart_pos[1])] = 255
            #score_list.append(score)
            #predicted_score_list.append(predicted_score)

            print(f'P/T: {predicted_score}/{score}')

            # Log every 5 batches.
            if i % 100 == 0:
                print(f"Progress: {(i + 1)}")

1/1 [==============================] - 1s 1s/step
P/T: 20/20.0
1/1 [==============================] - 1s 731ms/step
P/T: 40/40.0
1/1 [==============================] - 1s 704ms/step
P/T: 100/100.0
1/1 [==============================] - 1s 749ms/step
P/T: 3/3.0
1/1 [==============================] - 1s 776ms/step
P/T: 4/4.0
1/1 [==============================] - 1s 724ms/step
P/T: 24/24.0
1/1 [==============================] - 1s 727ms/step
P/T: 0/0.0
1/1 [==============================] - 1s 787ms/step
P/T: 5/5.0
1/1 [==============================] - 1s 788ms/step
P/T: 25/25.0
1/1 [==============================] - 1s 726ms/step
P/T: 20/20.0
1/1 [==============================] - 1s 729ms/step
P/T: 32/32.0
1/1 [==============================] - 1s 795ms/step
P/T: 33/33.0
1/1 [==============================] - 1s 733ms/step
P/T: 5/5.0
1/1 [==============================] - 1s 701ms/step
P/T: 45/25.0
1/1 [==============================] - 1s 776ms/step
P/T: 39/39.0
1/1 [================

In [1]:
a = np.array(predicted_score_list)
b = np.array(score_list)
print(np.sum(a==b)/len(predicted_score_list[-150:]))

NameError: name 'np' is not defined

In [41]:
score_list = []
predicted_score_list = []
for i in range(len(dataframe)):
    if dataframe["img_folder"][i] in test:
        if 'DSC' in dataframe["img_name"][i]:
            group = 'test'
            score = dataframe['score'][i]
            predicted_score = 0

            image_path = f'/home/jacob-relle/Dokumente/AML Projekt/cropped_images/800/{group}/{dataframe["img_folder"][i]}/{dataframe["img_name"][i]}'

            #load the data to test and validate
            image = load_img(image_path)
            ring = ringNet.predict(image,verbose=0)
            ring = np.argmax(ring,axis=3)

            wedge = wedgeNet.predict(image,verbose=0)
            wedge = np.argmax(wedge,axis=3)
            wedge[wedge==21] = 25
            
            total = ring*wedge
            total[total == 75] = 50
            
            dartNet_result = dartNet(image_path)
            darts = dartNet_result.xyxyn[0][:,:-1].numpy()
            for j in range(min(darts.shape[0],3)):
                if darts[j,-1] >= 0.557:
                    dart_pos = (int(400*(darts[j,3] + darts[j,1]) /2) , int(400*(darts[j,2] + darts[j,0]) /2))
                    predicted_score += total[0,int(dart_pos[0]),int(dart_pos[1])]
                    total[0,int(dart_pos[0]),int(dart_pos[1])] = 255
            score_list.append(score)
            predicted_score_list.append(predicted_score)
            
            print(f'P/T: {predicted_score}/{score}')

P/T: 9/10.0
P/T: 36/42.0
P/T: 18/21.0
P/T: 13/18.0
P/T: 43/37.0
P/T: 54/58.0
P/T: 40/33.0
P/T: 39/64.0
P/T: 40/61.0
P/T: 25/11.0
P/T: 75/78.0
P/T: 30/33.0
P/T: 35/34.0
P/T: 23/15.0
P/T: 34/37.0
P/T: 32/36.0
P/T: 29/32.0
P/T: 32/35.0
P/T: 39/49.0
P/T: 24/30.0
P/T: 28/46.0
P/T: 35/65.0
P/T: 41/49.0
P/T: 34/31.0
P/T: 26/42.0
P/T: 54/38.0
P/T: 80/87.0
P/T: 21/31.0
P/T: 6/9.0
P/T: 6/16.0
P/T: 19/23.0
P/T: 25/42.0
P/T: 47/45.0
P/T: 19/33.0
P/T: 23/35.0


KeyboardInterrupt: 